# Исследование надежности заемщиков.

Заказчиком исследования является кредитный отдел банка. Его целью является установление зависимости своевременного исполнения заемщиками кредитных обязательств от их семейного положения и количества детей. Для исследования банком представлена статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга. Статистической моделью, в которой будут представлены анализируемые категории изберем таблицы, в которых сопоставим общее количество заемщиков по заданным категориям (семейное положение, количество детей, уровень дохода, цель кредитования) с количеством имеющих факты несвоевременного возврата кредитов, и посчитаем долю. Для наглядности составим таблицу, в которой в абсолютном выражении будет показано количество заемщиков, имеющих долги по кредитам, с учетом семейного положения и количества детей в совокупности.
    Перед построением таблиц проведем предобработку данных, включающую утранение аномальных значений, пропусков, дубликатов.   </div> 

### Шаг 1. Обзор данных

In [1]:
import pandas as pd
data = pd.read_csv('C:/Users/User/Downloads/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64


Пропущенные значения имеются только в данных по столбцам 'days_employed' и 'total_income',то есть об общем трудовом стаже и ежемесячном заработке.
Количество пропущенных зачений одинаково по обоим параметрам, составляет 2174 и допущено скорее всего одними и теми же потенциальными заемщиками.
Долю пропусков по столбцу 'days_employed' в процентах от общего количества можно вычислить следующим образом. Как мы узнали, по любому другому столбцу (за исключением двух указанных) пропущенные данные отсутствуют. Следовательно количество данных например по столбцу 'purpose' можно взять за 100 % и узнать какой % от них составляют пропущенные значения.
Для подсчета данных в столбцах 'days_employed' и 'purpose' используем метод count(), результат сохраняем в переменной days_employed_part. Вычисляем соотношение количства данных по стомбцу 'days_employed' к данным по столбцу 'purpose'.
Долю в процентах находим методом f', в котором указываем один знак после запятой. 
Результат выводим на экран.
</div>

In [3]:
days_employed_part = (data['days_employed'].count())/(data['purpose'].count())
print(f'{days_employed_part:.1%}') 

89.9%


 Доля пропущенных значений в данных о трудовом стаже составляет 11 %. Скорее всего эти данные и о ежемесячном заработке потенциальные заемщики умышленно не указали по нескольким причинам. 
     В числе основных могут быть: 
    -отсутствие официально оформленных трудовых отношений, 
    -низкий доход (как боязнь отказа в выдаче кредита),
    -реальное отсутствие стажа работы и заработка.
    Логично предположить, что наибольшая часть потенциальных заемщиков, не указавших данные о заработке и стаже, находятся в неоформленных трудовых отношениях и получают ежемесячный доход. Поскольку данные о ежемесячном заработке являются комилественными переменными  пропущенные значения в столбце 'total_income' возможно заполнить медианным значением для поиска которого применяем метод median(). 
    Медианное значение показателя дохода предпочтительнее среднего, поскольку доход нескольких лиц может на порядок отличаться от остальных и среднее значение будет некорректно. Медиана же представляет собой значение, по отношению к которому половина больше него, а вторая половина меньше. Медианное  значение сохраняем в переменной total_income_median и выводим его на экран.
   </div>

In [4]:
total_income_median = data['total_income'].median()
print(total_income_median)

145017.93753253992


Медианное значение  составляет 145017.93753253992.
    Заполнение пропусков осуществляем посредством метода fillna(). Для контроля корректности работы кода выводим на экран первые 20 строк по столбцу 'total_income' до его применения  и после. 


### Шаг 2.1 Заполнение пропусков

In [5]:
print(data['total_income'].head(20))
data['total_income'] = data['total_income'].fillna(total_income_median)

data['total_income'].head(20)


0     253875.639453
1     112080.014102
2     145885.952297
3     267628.550329
4     158616.077870
5     255763.565419
6     240525.971920
7     135823.934197
8      95856.832424
9     144425.938277
10    113943.491460
11     77069.234271
12              NaN
13    130458.228857
14    165127.911772
15    116820.904450
16    289202.704229
17     90410.586745
18     56823.777243
19    242831.107982
Name: total_income, dtype: float64


0     253875.639453
1     112080.014102
2     145885.952297
3     267628.550329
4     158616.077870
5     255763.565419
6     240525.971920
7     135823.934197
8      95856.832424
9     144425.938277
10    113943.491460
11     77069.234271
12    145017.937533
13    130458.228857
14    165127.911772
15    116820.904450
16    289202.704229
17     90410.586745
18     56823.777243
19    242831.107982
Name: total_income, dtype: float64

 Выведение на экран первых 20 строчек по столбцу ['total_income'] до и после применения метода fillna() подтвердило, что по строке 12 первоначальное значение NaN заменилось на медианное.
Также проверяем по каким столбцам остались пропуски методами isna() и sum().  


In [6]:
print(data.isna().sum())

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64


Проверка показала, что пропуски в столбце 'total_income' устранены и по условию задачи остались только в столбце 'days_employed'.  


### Шаг 2.2 Проверка данных на аномалии и исправления.

Для выявления иных аномалий в данных по столбцу 'days_employed' мы вывели на экран первые 20 строк.  



In [7]:
data['days_employed'].head(20)
         

0      -8437.673028
1      -4024.803754
2      -5623.422610
3      -4124.747207
4     340266.072047
5       -926.185831
6      -2879.202052
7       -152.779569
8      -6929.865299
9      -2188.756445
10     -4171.483647
11      -792.701887
12              NaN
13     -1846.641941
14     -1844.956182
15      -972.364419
16     -1719.934226
17     -2369.999720
18    400281.136913
19    -10038.818549
Name: days_employed, dtype: float64


Их изучение показало, что в столбце много значений со знаком минус (-). Здравый смысл подсказывает, что отрицательных значений по показателю стажа работы быть не может. По всей видимости знак минус перед количественными переменными появился либо ввиду программного сбоя, либо (с меньшей вероятностью) потенциальные заемщики, заполнявшие электронную форму ставили указанный знак в качестве тире перед указанием числового показателя.
Для получения абсолютных значений (модуля) числовых данных по всему столбцу  'days_employed' используем функцию abs(). Первые 20 строк выводим на экран.</div>



In [8]:
data['days_employed'] = abs(data['days_employed'])          
data['days_employed'].head(20)           

0       8437.673028
1       4024.803754
2       5623.422610
3       4124.747207
4     340266.072047
5        926.185831
6       2879.202052
7        152.779569
8       6929.865299
9       2188.756445
10      4171.483647
11       792.701887
12              NaN
13      1846.641941
14      1844.956182
15       972.364419
16      1719.934226
17      2369.999720
18    400281.136913
19     10038.818549
Name: days_employed, dtype: float64

Проверка показала, что значения по столбцу 'days_employed' перестали быть отрицательными. 
    Однако имеются пропуски данных, обозначенные как NaN которые заполняем медианным значением по аналогии с работой со столбцом 'total_income' и проверяем правильность работы кода путем контроля изменения значения по строке 12 столбца 'days_employed'.  


In [9]:
days_employed_median = data['days_employed'].median()
print(days_employed_median)
data['days_employed'] = data['days_employed'].fillna(days_employed_median)
data['days_employed'].head(20)

2194.220566878695


0       8437.673028
1       4024.803754
2       5623.422610
3       4124.747207
4     340266.072047
5        926.185831
6       2879.202052
7        152.779569
8       6929.865299
9       2188.756445
10      4171.483647
11       792.701887
12      2194.220567
13      1846.641941
14      1844.956182
15       972.364419
16      1719.934226
17      2369.999720
18    400281.136913
19     10038.818549
Name: days_employed, dtype: float64

In [10]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Столбцы 'children','education','family_status','gender','income_type', 'purpose' проверены далее и проведена их обработка. Не изученными и не оцененными были столбцы 'dob_years', 'total_income', 'debt', 'education_id', 'family_status_id'. Как было указано, пропущенных значений в эих столбцах не установлено. Изучим их на предмет наличия аномальных значений методом value_counts(). 


 Поскольку аномалии, как правило, встречаются ближе к концу данных по столбцу 'total_income' выведем последние 20 строк.</div>

In [11]:
print(data['total_income'].value_counts().tail(20))

165029.871363    1
152410.919231    1
323466.323030    1
111976.479710    1
135607.236400    1
204204.182683    1
262517.623178    1
155107.752145    1
160528.666287    1
160885.050715    1
100180.178296    1
155567.245644    1
124357.256315    1
175057.266090    1
101516.604975    1
239154.168013    1
165009.733021    1
94270.049769     1
68166.012050     1
189255.286637    1
Name: total_income, dtype: int64


In [12]:
data['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [13]:
data['education_id'].value_counts()

1    15233
0     5260
2      744
3      282
4        6
Name: education_id, dtype: int64

In [14]:
data['family_status_id'].value_counts()

0    12380
1     4177
4     2813
3     1195
2      960
Name: family_status_id, dtype: int64

In [15]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

 В столбце 'dob_years' установлено, что возраст заемщиков 101 раз указан как 0, что несомненно является ошибкой. Однако, поскольку это не пропущенное значение и возраст не является анализируемой категорией, возможно это значение оставить в исходном виде, ничем не заменяя его и не удаляя. </div>

### Шаг 2.3. Изменение типов данных.

In [16]:

data['total_income'] = data['total_income'].astype('int')#'
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  int32  
 11  purpose           21525 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


 Методом astype() мы изменили тип данных в столбце 'total_income' на integer и проверили полученный результат, вызвав метод info().Данные по этому столбцу изменились на цельночисленные.

### Шаг 2.4. Удаление дубликатов.

 Методом duplicated().sum() находим количество явных дубликатов - полностью повторяющихся строк.

In [17]:
data.duplicated().sum()

54

 
Проверяем наличие дубликатов в отдельных столбцах типа object.

In [18]:
data['education'].value_counts()

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

 В столбце 'education' имеются неявные дубликаты: одни и те же названия, записанные разным регистром, что является одной из причин появления неявных дубликатов. Приводим их к одному нижнему регистру методом str.lower().

In [19]:
data['education'] = data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [20]:
data['gender'].value_counts()

F      14236
M       7288
XNA        1
Name: gender, dtype: int64

 В столбце 'gender' найдено значение  XNA, которое возможно является пропуском или какой-то аномалией. Это значение единственное, на статистику влияния не окажет, тем более то показатель гендерной принадлежности по условию задачи исследованию не подлежит. В этой связи его можно заменить наибольшим показателем в этом столбце c использованием атрибута loc.

In [21]:
for row in data['gender']:             
    try:
        data.loc[data['gender'] == 'XNA', 'gender'] = 'F' 
    except:    
        print('Некорректные строки') 

In [22]:
print(data['gender'].value_counts())

F    14237
M     7288
Name: gender, dtype: int64


 В столбце 'children' установлены аномальные значения: 47 заемшщиков, у которых количество детей указано с отрицательным знаком(-1), что в принципе быть не может и скорее всего потенциальные заемщики хотели указать по 1 ребенку. Еще 76 потенциальных заемщиков указали о наличии 20 детей, что также маловероятно и логично предположить, что имелось ввиду по 2 ребенка.   

In [23]:
print(data['children'].value_counts())

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


 Для изменения отрицательных значений на абсолютное применяем атрибут abs(), 20 детей заменяем на 2 атрибутом loc.   

In [24]:
data['children'] = abs(data['children'])
data.loc[data['children'] == 20,'children'] = 2
data['children'].value_counts()


0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [25]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

In [26]:
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

 В столбце 'family_status' значение  Не женат / не замужем записано с применением разного регистра. Дубликатом оно здесь не является, но для единообразия также приведем все наименования в этом столбце к одному нижнему регистру методом str.lower().

In [27]:
data['family_status'] = data['family_status'].str.lower()
data['family_status'].value_counts()

женат / замужем          12380
гражданский брак          4177
не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64

In [28]:
data['purpose'].value_counts()

свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
покупка жилья                             647
жилье                                     647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться высшим образованием      

 В столбцах 'income_type', 'family_status' пропусков, дубликатов и иных аномалий не обнаружено. 
В столбце 'purpose' встречается много значений, которые можно отнести к дубликатам, но поскольку это не количественные переменные, то их лучше категоризировать, что будет сделано по условию задачи в разделе 2.7. 

 Теперь еще раз посчитаем количество явных дубликатов.  

In [29]:
data.duplicated().sum()

71

 Избавляемся от дубликатов методом drop_duplicates().reset_index(drop=True) с заменой индексации и проверяем фактическое их устранение.

In [30]:
data = data.drop_duplicates().reset_index(drop=True)
data.duplicated().sum()

0

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [31]:
data_education = data[['education','education_id']]
data_education = data_education.drop_duplicates().reset_index(drop=True)
print(data_education)

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4


In [32]:
data_family_status = data[['family_status','family_status_id']]
data_family_status = data_family_status.drop_duplicates().reset_index(drop=True)
print(data_family_status)

           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  не женат / не замужем                 4


 На основе исходного датафрейма мы создали два других со столбцами 'education','education_id' и'family_status','family_status_id', сохранив их в переменных data_education и data_family_status. 
    В соответствии с условием задачи из исходного датафрейма удалили столбец 'education' методом drop(),axis со значением 1 указал на применение этого метода ко всем строкам. Столбец 'family_status' вопреки условию задачи оставили, так как в последующем, при построении таблицы зависимости своевременного возврата кредита от семейного положения и количества детей он представляется более наглядным, чем его оцифрованные значения.

In [33]:
data = data.drop(['education'],axis=1) 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21454 non-null  int64  
 1   days_employed     21454 non-null  float64
 2   dob_years         21454 non-null  int64  
 3   education_id      21454 non-null  int64  
 4   family_status     21454 non-null  object 
 5   family_status_id  21454 non-null  int64  
 6   gender            21454 non-null  object 
 7   income_type       21454 non-null  object 
 8   debt              21454 non-null  int64  
 9   total_income      21454 non-null  int32  
 10  purpose           21454 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(4)
memory usage: 1.7+ MB


### Шаг 2.6. Категоризация дохода.

 Данные о доходах заемщиков многочисленны и разнообразны. Чтобы установить взаимосвязь между доходами заемщиков и их дисциплинированностью в плане своевременной оплаты кредитов необходимо сведения о доходах категоризировать в соответсвии с параметрами, установленными в условии задачи. Для этого мы создадим собственную функцию total_group в качестве аргумента которой передадим данные столбца 'total_income'. Указанная функция возвращает значение категории, определенное в теле функции в зависимости от дохода. Методом apply() создаем в data столбец 'total_income_category', в котором в результате работы функции total_group отобразится категория дохода в соответствии с заданными параметрами.

In [34]:
def total_group(total_income):
    if total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000: 
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if  200001 <= total_income <= 1000000: 
        return 'B'
    if total_income >= 1000001:
        return 'A'
data['total_income_category'] = data['total_income'].apply(total_group)
print(data['total_income_category'].value_counts())

C    16016
B     5041
D      350
A       25
E       22
Name: total_income_category, dtype: int64


 Проверка показала, что функция работает. С помощью метода info() убедимся в появлении столбца 'total_income_category'.

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21454 non-null  int64  
 1   days_employed          21454 non-null  float64
 2   dob_years              21454 non-null  int64  
 3   education_id           21454 non-null  int64  
 4   family_status          21454 non-null  object 
 5   family_status_id       21454 non-null  int64  
 6   gender                 21454 non-null  object 
 7   income_type            21454 non-null  object 
 8   debt                   21454 non-null  int64  
 9   total_income           21454 non-null  int32  
 10  purpose                21454 non-null  object 
 11  total_income_category  21454 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(5)
memory usage: 1.9+ MB


### Шаг 2.7. Категоризация целей кредита.

 Для оптимизации анализа зависимости платежеспособности заемщиков от целей кредита последние также категоризируем. Для этой цели создадим функцию purpose_category, в качестве аргумента которой передадим данные столбца 'purpose'. В теле функции пропишем условие, согласно которому при наличии определенных сочетаний букв функция будет возвращать обобщенные наименования четырех категорий.

In [36]:
def purpose_category (purpose):
    if 'авто' in purpose:
        return 'операция с автомобилем'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образован' in purpose:
        return 'получение образования'
    if 'недвижим' or 'жиль' in purpose:
        return 'операции с недвижимостью'
data['purpose_category'] = data['purpose'].apply(purpose_category)
print(data['purpose_category'].value_counts())

операции с недвижимостью    10811
операция с автомобилем       4306
получение образования        4013
проведение свадьбы           2324
Name: purpose_category, dtype: int64


 Результатом работы функции методом apply() мы дополнили исходный датафрейм в столбце 'purpose_category'. Проверим, вызвав метод info()

In [37]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   children               21454 non-null  int64  
 1   days_employed          21454 non-null  float64
 2   dob_years              21454 non-null  int64  
 3   education_id           21454 non-null  int64  
 4   family_status          21454 non-null  object 
 5   family_status_id       21454 non-null  int64  
 6   gender                 21454 non-null  object 
 7   income_type            21454 non-null  object 
 8   debt                   21454 non-null  int64  
 9   total_income           21454 non-null  int32  
 10  purpose                21454 non-null  object 
 11  total_income_category  21454 non-null  object 
 12  purpose_category       21454 non-null  object 
dtypes: float64(1), int32(1), int64(5), object(6)
memory usage: 2.0+ MB


### Ответы на вопросы.

 Для вывода о зависимости возврата кредита в срок от таких параметров потенциальных заемщиков как количество детей, семейного положения, уровня дохода и целей кредита построим таблицы с применением комбинации методов groupby по столбцам 'children', 'family_status', 'total_income_category', 'purpose_category' с методом agg по столбцу 'debt' с агрегирующими функциями sum() и count(). Для наглядности в каждой из таблиц создадим столбец 'debt_share' в котором посчитаем долю невозвращенных кредитов от их общего числа по каждой анализируемой категории.

In [38]:
data.pivot_table(index="children", values="debt", aggfunc=['sum','count','mean'])

,sum,count,mean
,debt,debt,debt
children,,,
0,1063,14091,0.075438
1,445,4855,0.091658
2,202,2128,0.094925
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


##### Вывод 1:
Таблица в которой соотнесены количество фактов несвоевременного возврата кредитов и количества детей, имеющихся у потенциальных заемщиков свидетельствует о том, что отсутствуют факты просрочки по кредитным платежам у граждан с 5 детьми. Однако и случаи их обращения за кредитами единичны. 
Отсутствие детей, а также их наличие в семье в количестве от 1 до 4 незначительно влияют на своевременность воозврата кредитов. По каждой из указанных категорий несвоевременно возвращают кредиты от 7.5 % до 9.7 %. Наиболее благонадежными являются лица, не имеющие детей. Только 7.5 % из них не возвращают кредиты вовремя. Наименее дисциплинированы заемщики с 4 детьми. Доля их невозвратов составляет 9.7 %. Лица, имеющие от 1 до 3 детей несвоевременно возвращают кредиты в 8 - 9.5 % случаев. 

In [39]:
data.pivot_table(index='family_status', values="debt", aggfunc=['sum','count','mean'])

,sum,count,mean
,debt,debt,debt
family_status,,,
в разводе,85,1195,0.071130
вдовец / вдова,63,959,0.065693
гражданский брак,388,4151,0.093471
женат / замужем,931,12339,0.075452
не женат / не замужем,274,2810,0.097509


 ##### Вывод 2:
Таблица, в которой соотнесены семейное положение заемщиков и частота несвоевременного возврата ими кредитов демонстрирует, что наиболее дисциплинированны в финансовом отношении заемщики, являющиеся вдовцами/вдовами (6.5 %), а также разведенные и состоящие в официально зарегистрированных семейных отношениях (7.1 % и 7.5 % соответственно). Не связанные семейными обязательствами лица чаще других категорий опаздывают с платежами по кредитам. Доля неженатых (незамужних) составляет 9.7 %, состоящих в гражданском браке - 9.3 %.

 Увидеть в абсолютном выражении число заемщиков, имеющих факты несвоевременного возврата кредитов, в совокупности по их семейному положению и количеству детей можно в таблице, построенной с применением метода pivot_table, в котором сгруппируем данные по столбцу'children', в качестве столбца, по значениям которого произойдет группировка,  укажем 'family_status', значение, по которым будем смотреть таблицу возьмем 'debt' с функцией 'sum'.

In [40]:
data_pivot_children = data.pivot_table(index=['children'], columns= 'family_status', values='debt', aggfunc= sum) 
display(data_pivot_children)

family_status,в разводе,вдовец / вдова,гражданский брак,женат / замужем,не женат / не замужем
children,,,,,
0,55.0,53.0,229.0,516.0,210.0
1,21.0,7.0,118.0,247.0,52.0
2,8.0,3.0,33.0,148.0,10.0
3,1.0,0.0,8.0,17.0,1.0
4,0.0,0.0,0.0,3.0,1.0
5,NaN,NaN,0.0,0.0,NaN


In [41]:
data.pivot_table(index='total_income_category', values="debt", aggfunc=['sum','count','mean'])

,sum,count,mean
,debt,debt,debt
total_income_category,,,
A,2,25,0.080000
B,356,5041,0.070621
C,1360,16016,0.084915
D,21,350,0.060000
E,2,22,0.090909


##### Вывод 3:
При анализе зависимости финансовой благонадежности клиентов от их дохода следует отметить, что при более высоком проценте должников среди заемщиков, имеющих самые высокие и самые низкие доходы (категории A и E соответственно) их абсолютное количество единично (по 2 в каждой категории), как и количество лиц, каждой из эих категорий, взявших кредиты (25 и 22 соответственно, что составляет 0.1 %). Статистика по таким лицам не представляется показательной.  В числе других категорий  наиболее часто (8.4 % случаев) несвоевременно возвращают кредиты заемщики из категории С (с суммой дохода от 50001 до 200000 руб.). Заемщики из категории B (с суммой дохода от 200001 до 1000000 руб.) являются вторыми по долгам, из них 7 % несвоевременно возвращают кредиты. Заемщики, отнесенные по сумме дохода к категории D ( от 30001 до 50000 руб) являются наиболее благонадежными (6 % несвоевременных кредитных платежей). 
       

In [42]:
data.pivot_table(index='purpose_category', values="debt", aggfunc=['sum','count','mean'])

,sum,count,mean
,debt,debt,debt
purpose_category,,,
операции с недвижимостью,782,10811,0.072334
операция с автомобилем,403,4306,0.093590
получение образования,370,4013,0.092200
проведение свадьбы,186,2324,0.080034


##### Вывод 4:
Из таблицы следует, что наиболее подвержены несвоевременной оплате кредита заемщики, чьей целью были операции с автомобилем и получение образования. Наименьшие риски невозврата у тех, кто брал кредиты на операции с недвижимостью и проведение свадьбы. 

## Общий вывод:


   
По заказу кредитного отдела банка мы провели исследование зависимости своевременного исполнения заемщиками кредитных обязательств от их семейного положения и количества детей.Результаты исследования необходимы для построения модели кредитного скоринга.Для исследования банком представлена статистика о платёжеспособности клиентов.

 Статистической моделью, в которой представлены анализируемые категории, избраны таблицы, в которых сопоставлно общее количество заемщиков по заданным категориям (семейное положение, количество детей, уровень дохода, цель кредитования) с количеством имеющих факты несвоевременного возврата кредитов, и посчитана доля. Для наглядности составлена таблица, в которой в абсолютном выражении показано количество заемщиков, имеющих долги по кредитам, с учетом семейного положения и количества детей в совокупности. 
    Перед построением таблиц проведены ознакомление с общей информацией о данных, их предобработка, включая устранение аномальных значений, пропусков, дубликатов.  
    
   Исследование зависимости семейного положения клиентов, а также количества детей на факт погашения кредитов в срок показало следующее. 
   
   В базе данных, представленных банком имеются сведения о заемщиках с количеством детей от 0 до 5. Установлено, что отсутствие детей, а также их наличие в семье в количестве от 1 до 4 влияют на своевременность воозврата кредитов незначительно. По каждой из указанных категорий несвоевременно возвращают кредиты от 7.5 % до 9.7 %. Наиболее благонадежными являются лица, не имеющие детей ( 7.5 %), наименее - заемщики с 4 детьми (9.7 %).
   
   Своевременность платежей и семейное положение заемщиков немногим более взаимозависимы. Наиболее дисциплинированны в финансовом отношении заемщики, являющиеся вдовцами/вдовами (6.5 %), а также разведенные и состоящие в официально зарегистрированных семейных отношениях (7.1 % и 7.5 % соответственно). Доля связанных семейными обязательствами лиц превышает 9 % (неженатые (незамужние) - 9.7 %, состоящие в гражданском браке - 9.3 %).
   
   Наиболее часто (8.4 % случаев) несвоевременно возвращают кредиты заемщики с суммой дохода от 50001 до 200000 руб. Заемщики с доходом от 30001 до 50000 руб. являются более благонадежными (6 % просроченных возвратов).
   Зависимость несвоевременной оплаты кредитов от их целей менее вариативна (в пределах 2 %). Наименьшие риски невозврата у тех, кто брал кредиты на операции с недвижимостью и проведение свадьбы.
   
   
   На основании данных всех четырех таблиц статистическое лицо наиболее и наименее благонадежных заемщиков выглядит следующим образом:
   
   наименее благонадежный заемщик имеет доход не женат (не замужем), имеет детей, имеет доход от 50001 до 200000 руб. и берет кредит на автомобиль;
   
   наиболее благонадежный заемщик получает от 30001 до 50000 руб., берет кредит на сделки с недвижимостью является вдовцом и не имеет детей.
   
   При формировании модели кредитного скоринга заказчику следует больше обращать внимание на семейное положение потенциальных заемщиков как более вариативную категорию.В указанной связи предпочтение следует отдавать тем, кто состоит в официально зарегистрированных семейных отношениях, в разводе, либо является вдовцом (вдовой). Также следует принимать во внимание сумму дохода потенциальных заемщиков, которая не прямо пропорциональна их финансовой благонадежности. Наиболее благонадежными являются граждане, получающие доход в размере от от 30001 до 50000 руб. и от 200001 до 1000000 руб. Вариативность таких категорий как количество детей и цель кредитования незначительно влияет на платежеспособность клиентов банка и заслуживает меньшего по сравнению с указанными выше параметрами внимания.